# Imports

In [ ]:
# Python standard library
import sys
import csv

# Scientific computing
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage

# Program specific
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/rp/')
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi/')
import resistive_pulse as rp
import rp_file
import optical_imaging as oi
import oi_file


# Jupyter
from IPython.display import HTML

# Load data

In [ ]:
date = '10-10-2017/'
particle = '293-T_0/'
channel = '10-20-10_3/'
file_index = '0'

base_path = '/home/prestonh/Desktop/Research/cancer_cells/data/'

oi_vid_file_path = base_path + date + particle + channel + 'oi/bin/test_camera_' + file_index
oi_events_file_path = base_path + date + particle + channel + 'oi/events/test_camera_' + file_index + '_events.json'

In [ ]:
# Load video
res_x = 384
res_y = 112
oi_fps = 100000

oi_vid = oi_file.Video(oi_vid_file_path, res_x, res_y, oi_fps)

# Load events
oi_events = oi_file.open_event_file_json(oi_events_file_path)

In [ ]:
print len(oi_events)

##### Load ellipses

In [ ]:
oi_ellipse_base_path = '../data/'
oi_ellipse_path = oi_ellipse_base_path + date + particle + channel + file_index + '/ellipses'

ellipsess = []


with open(oi_ellipse_path, 'r') as file_handle:
    file_reader = csv.reader(file_handle, delimiter = '\t')
    
    # Skip header
    header_length = 2
    for i in range(header_length):
        next(file_reader)
        
        
    for row in file_reader:
        
        if row[0] == 'event #':
            # New event
            ellipsess.append([])
            continue
            
        else:
            ellipse = []
            for ele in row:
                try:
                    ellipse.append(float(ele))
                except:
                    ellipse.append(0)
                    
            ellipsess[-1].append(ellipse)

# Load template and create stage

In [ ]:
template_index = 100

template_frame = oi_vid.get_frame(template_index)

plt.imshow(template_frame, cmap = 'gray', origin = 'lower')
plt.show()

In [ ]:
reload(oi)
template_frame = oi_vid.get_frame(template_index)

stage_file_path = base_path + date + particle + channel + 'oi/stage/stage_' + file_index
cs = oi.load_stage_file(stage_file_path)

c0 = cs[0]
c1 = cs[1]
c2 = cs[2]
c3 = cs[3]

oi_stage = oi.Stage(template_frame, c0, c1, c2, c3)
oi_stage.plot_stage()

# Begin filtering

In [ ]:
filtering_steps = {}

### Channel enter exit based filtering

In [ ]:
keep_indices_enter_exit = []

for i, oi_event in enumerate(oi_events):
    try:
        xs = []
        ys = []

        for j, ellipse in enumerate(ellipsess[i]):
            xs.append(ellipse[1])
            ys.append(ellipse[2])

        xcs, ycs = oi_stage.get_channel_coordinates(xs, ys)
        xcs = oi_stage.pixels_to_meters(xcs)
        ycs = oi_stage.pixels_to_meters(ycs)

        if xcs.min() < 0 and xcs.max() > oi_stage._length_microns:
            keep_indices_enter_exit.append(i)
            
    except:
        # Catch-all: Do not append this element to the keep_indices_enter_exit list;
        #            This will prevent it from being analyzed.
        print 'failed on', i
        pass

In [ ]:
filtering_steps['channel enter exit'] = [True]

print 'length before', len(oi_events), 'length after', len(keep_indices_enter_exit)
print
print 'events filtered by entrance/exit', [i for i in range(len(ellipsess)) if i not in keep_indices_enter_exit]

In [ ]:
oi_index = 0
reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf, oi_index).to_html5_video())

## Geometry based filtering

In [ ]:
# Define left and right bounds that particle must be between for fitting its radius and aspect
xc_left = -50
xc_right = 0

yc_top = 50
yc_bottom = -50



effective_radii = []
effective_radius_max = 30

aspect_ratios = []
aspect_ratio_max = 3



for i, oi_event in enumerate(oi_events):
  
    aspect_ratio = 0
    effective_radius = 0
    for j, ellipse in enumerate(ellipsess[i]):
        
        x = ellipse[1]
        y = ellipse[2]
        
        xc, yc = oi_stage.get_channel_coordinates(x, y)
        xc = oi_stage.pixels_to_meters(xc)
        yc = oi_stage.pixels_to_meters(yc)
        
        
        
        
        if xc > xc_left and xc < xc_right and yc > yc_bottom and yc < yc_top:
            
            #plt.imshow(oi_vid.get_frame(oi_event._detections[int(ellipse[0])]._tf), cmap = 'gray', origin = 'lower')
            #plt.show()
            
            ax0 = ellipse[3]
            ax1 = ellipse[4]
            
            a = oi_stage.pixels_to_meters(ax0)
            b = oi_stage.pixels_to_meters(ax1)
            
            aspect_ratio = a/b

            effective_radius = np.sqrt(a*b)
        
            break
            
            
    # Catch bad values
    if effective_radius > effective_radius_max or np.isnan(effective_radius) or np.isinf(effective_radius):
        print i
        effective_radius = 0
    
    
    
    if np.isnan(aspect_ratio) or np.isinf(aspect_ratio) or aspect_ratio > aspect_ratio_max:
        aspect_ratio = 0
    
    
        
    # Append values to lists        
    effective_radii.append(effective_radius)
    aspect_ratios.append(aspect_ratio)
    
            
            
plt.hist(effective_radii, bins = 50)
plt.title('effective radius')
plt.grid()
plt.show()
            
plt.hist(aspect_ratios, bins = 50)
plt.title('aspect ratios')
plt.grid()
plt.show()      

## Filter size

In [ ]:
# Filter size
radius_min = 5
radius_max = 11

filtering_steps['effective radius'] = [radius_min, radius_max]

keep_indices_radius = [i for i in range(len(effective_radii)) if ((effective_radii[i] > radius_min) and (effective_radii[i] < radius_max))]


# Filter aspect

aspect_min = 0.75
aspect_max = 1.25

filtering_steps['aspect ratio'] = [aspect_min, aspect_max]

keep_indices_aspect = [i for i in range(len(aspect_ratios)) if ((aspect_ratios[i] > aspect_min) and (aspect_ratios[i] < aspect_max))]



# Histograms
plt.hist(np.array(effective_radii)[keep_indices_radius], bins = 20)
plt.title('effective radius')
plt.show()
            
plt.hist(np.array(aspect_ratios)[keep_indices_aspect], bins = 20)
plt.title('aspect ratios')
plt.show()  




# Print

print ('radius filtering ' + str(len(oi_events) - len(keep_indices_radius)) + '/' + str(len(oi_events)))

print

print 'indexes sorted by radius\n', [ele for ele in np.argsort(effective_radii) if ele in keep_indices_radius]

print 

print ('aspect ratio filtering ' + str(len(oi_events) - len(keep_indices_aspect)) + '/' + str(len(oi_events)))

print

print 'indexes sorted by aspect\n', [ele for ele in np.argsort(aspect_ratios) if ele in keep_indices_aspect]

print


print ('events filtered by radius (sorted by size):')
effective_radii_tuple = [(i, effective_radii[i]) for i in range(len(effective_radii)) if i not in keep_indices_radius]
print([tuple[0] for tuple in sorted(effective_radii_tuple, key = lambda effective_radius_tuple: effective_radius_tuple[1])])

print

print ('events filtered by aspect:')
print([i for i in range(len(oi_events)) if i not in keep_indices_aspect])

print

print('events filtered out by size but kept by aspect:')
print([i for i in range(len(oi_events)) if (i in keep_indices_aspect) and (i not in keep_indices_radius)])

print


print('events filtered out by aspect but kept by size:')
print([i for i in range(len(oi_events)) if (i not in keep_indices_aspect) and (i in keep_indices_radius)])

print


    


In [ ]:
oi_index = 4
print 'radius:', effective_radii[oi_index]
print 'aspect:', aspect_ratios[oi_index]

oi_event = oi_events[oi_index]
ts = oi_event.get_tf()
t = int((ts[-1] + ts[0])/2.)
detection_index = t - ts[0]
detection = oi_event._detections[detection_index]
x = detection._px
y = detection._py
frame = oi_vid.get_frame(t)
plt.scatter(x, y, marker = 'x', s = 100, color = 'red', lw = 0.5)

plt.imshow(frame, cmap = 'gray', origin = 'lower')
plt.show()

In [ ]:
print oi_stage._c0
print oi_stage._c1
print oi_stage._c2
print oi_stage._c3

In [ ]:
#oi_index = 799
print 'radius = ', effective_radii[oi_index]
print 'aspect = ', aspect_ratios[oi_index]
reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf, oi_fps).to_html5_video())

### y-based filtering

In [ ]:
# Central cavity y-based filtering

yc_middles = []
yc_failed_indices = []

# Set yc_middle_min and max... These are bounds that we do not accept events outside of. Usually if there is
# a really bad fit, the detected yc value could be something like -60. This is unphysical, so we 
# manually introduce yc_middle_min and yc_middle_max to filter out unphysical values.
yc_middle_min = -25
yc_middle_max = 25

for i, oi_event in enumerate(oi_events):
    
    try:
        # Calculation sometimes fails, put in a 'try' block
        
        
        # Get positions of ellipses and convert to channel micron units
        xs = [ellipse[1] for ellipse in ellipsess[i]]
        ys = [ellipse[2] for ellipse in ellipsess[i]]
    
        
        xcs, ycs = oi_stage.get_channel_coordinates(xs, ys)
        xcs = oi_stage.pixels_to_meters(xcs)
        ycs = oi_stage.pixels_to_meters(ycs)
        

        # Interpolate the indices of x-positions
        is_interp = scipy.interpolate.interp1d(xcs, range(len(xcs)))
    
    
        # Get the index closest to the middle of the channel
        i_middle = int(is_interp(oi_stage._length_microns/2.))
        
        
        
        yc_middle = ycs[i_middle]
        
        
        if yc_middle > yc_middle_max or yc_middle < yc_middle_min:
            # The determined yc_middle is outside the manual bounds we set above; raise a ValueError
            # to exit try statement and go to except.
            print 'value error'
            raise ValueError
    
        yc_middles.append(ycs[i_middle])
        
    except:
        # Calculation failed, add the index to the yc_failed_indices which will automatically be rejected
        # later on.
        print 'failed', i
        yc_failed_indices.append(i)
        yc_middles.append(-500)
    



    
# Apply offset to y so centered at 0... only count the successes
yc_middles = np.array(yc_middles)
yc_success_indices = np.array([i for i in range(len(oi_events)) if i not in yc_failed_indices])
offset = -(np.max(yc_middles[yc_success_indices]) - np.abs(np.min(yc_middles[yc_success_indices])))/2.
yc_middles = np.array(yc_middles)
yc_middles = yc_middles + offset



# Plot hist of all ys
plt.hist(yc_middles[yc_success_indices], bins = 20)
plt.xlabel('yc')
plt.show()



In [ ]:
yc_threshold = 3
filtering_steps['yc cavity threshold'] = [yc_threshold]

keep_indices_y = np.where(np.abs(yc_middles) <= yc_threshold)[0]
keep_indices_y = [keep_index_y for keep_index_y in keep_indices_y if keep_index_y not in yc_failed_indices]

print 'length before', len(oi_events), 'length after', len(keep_indices_y)
print
print 'events filtered by y', [i for i in range(len(ellipsess)) if i not in keep_indices_y]
print
print 'events kept by y', [i for i in range(len(ellipsess)) if i in keep_indices_y]

In [ ]:
oi_index = 1418
print 'radius', effective_radii[oi_index]
print 'aspect', aspect_ratios[oi_index]
print 'yc middle', yc_middles[oi_index]
reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf, oi_index).to_html5_video())

## Manual filtering
- Inspect the events that haven't been filtered out yet to make sure they are good.
- Add undesirable events to the 'manual_remove_indices' list

In [ ]:
keep_indices = [i for i in range(len(oi_events)) if ((i in keep_indices_enter_exit)\
                                                     and (i in keep_indices_radius)\
                                                     and (i in keep_indices_aspect)\
                                                     and (i in keep_indices_y))]

print 'keeping', len(keep_indices), '/', len(ellipsess), 'events before manual removal'
print 'events to be kept:', keep_indices

##### Inspect frame

In [ ]:
oi_index = 50
oi_event = oi_events[oi_index]
detection_index = int(len(oi_event._detections)/2.)
t = oi_event._detections[detection_index]._tf
frame = oi_vid.get_frame(t)
plt.imshow(frame, cmap = 'gray', origin = 'lower')


plt.scatter(oi_event._detections[detection_index]._px, oi_event._detections[detection_index]._py,\
            marker = 'x', c = 'red', lw = 3, s = 25)
plt.show()

##### Inspect video

In [ ]:
oi_index = 212

print 'radius:', effective_radii[oi_index]
print 'aspect:', aspect_ratios[oi_index]
print 'yc:', yc_middles[oi_index]

reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf, oi_index).to_html5_video())

In [ ]:
manual_remove_indices = []

filtering_steps['manual removal'] = [len(manual_remove_indices)]

print 'manually removing indices', manual_remove_indices

keep_indices_manual = [i for i in range(len(oi_events)) if i not in manual_remove_indices]

## Filtering step

In [ ]:
keep_indices = [i for i in range(len(oi_events)) if ((i in keep_indices_enter_exit)\
                                                     and (i in keep_indices_radius)\
                                                     and (i in keep_indices_aspect)\
                                                     and (i in keep_indices_y)\
                                                    and (i in keep_indices_manual))]


print(keep_indices)
print('filtering ' + str(len(oi_events) - len(keep_indices)) + ' out of ' + str(len(oi_events)) + ' events')

# Save
- Save the filtering parameters as well as the indices of the events to be kept

### Save filtering parameters and indices

In [ ]:
output_file_path = '../data/' + date + particle + channel + file_index + '/filter'
print 'saving to ', output_file_path


with open(output_file_path, 'w') as file_handle:
    file_writer = csv.writer(file_handle, delimiter = '\t')
    
    
    # Write filtering steps
    for filtering_step in filtering_steps:
        file_writer.writerow(['filter step', filtering_step, filtering_steps[filtering_step]])
        
    file_writer.writerow(['events'] + keep_indices)